# Importing Data

In [1]:
import os
import cv2
import glob
import time
import random
import warnings

import numpy as np
import pandas as pd


from sklearn import svm
from sklearn import tree
from sklearn import impute
from sklearn import metrics
from sklearn import ensemble
from sklearn import linear_model
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)



# Loading Data 

In [2]:
submission = pd.read_csv("sample_solution.csv")

In [3]:
train_data = pd.read_csv("train.csv").drop(columns = ["id"])
test_data = pd.read_csv("test.csv").drop(columns = ["id"])

N_FOLDS = 5
target = 'claim'
TARGET = train_data['claim']

train_data.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0.108590,0.004314,-37.566000,0.017364,0.289150,-10.251000,135.120000,168900.000000,399240000000000.000000,86.489000,0.598810,1423200000.000000,0.272400,9.455600,-0.050305,1938.300000,8.633100,4.060700,26.867000,-1.180000,10961.000000,1.539700,135.320000,-1.496500,440.080000,2590100000000.000000,2194200000.000000,2968800.000000,0.001431,13.327000,0.750500,18509.000000,146820.000000,-0.000276,10906000000000000.000000,1705.400000,414.290000,3.539200,1888.000000,0.968930,18.388000,-0.001583,7.705900,5.932500,0.025693,4.560400,0.611220,10.795000,0.341930,0.235010,nan,5237.700000,1.296100,163.660000,0.403780,0.188600,-0.001446,-0.354160,6.643200,0.305340,0.514020,1907300000.000000,29.861000,0.965010,1797.200000,72.178000,108.620000,1.979900,1.290700,0.995190,1.322800,827.340000,777990000000000.000000,41299000000.000000,0.006994,6.983500,43956.000000,1978.200000,5.508400,-0.001081,6.124400,123180000000.000000,275.920000,5308500.000000,1704.000000,50224000000.000000,53.398000,-2.201200,6871.000000,3.886200,-0.005580,5252.100000,166.690000,1.607400,0.665340,7768.900000,0.996620,112570000000.000000,2.243200,0.934160,0.650560,94569.000000,21.471000,8214.100000,0.288010,0.097826,0.001071,1412400000.000000,0.110930,-12.228000,1.748200,1.909600,-7.115700,4378.800000,1.209600,861340000000000.000000,140.100000,1.017700,1
1,0.100900,0.299610,11822.000000,0.276500,0.459700,-0.837330,1721.900000,119810.000000,3874100000000000.000000,9953.600000,1.209300,3334100000.000000,0.286310,-0.012858,-0.019912,10.284000,6.187200,1.041900,4.640400,31.877000,123620.000000,1.395100,125.810000,1.198900,136.450000,9098100000.000000,40041000000.000000,1564000.000000,0.000204,3.107400,1.503300,238000.000000,21440.000000,-0.001344,30794000000000000.000000,229.100000,844.820000,1.468000,4726.500000,0.915380,-1.532100,0.982600,7.111200,2.079700,0.042321,4.252300,0.418710,5.449900,0.012737,0.386470,7.308200,283.210000,-0.925520,140.800000,0.247390,-0.001656,-0.000975,-0.226290,2.424600,0.771470,0.011613,1803700000.000000,64.604000,0.262650,4455.000000,78.339000,745.510000,2.906900,1.482600,1.005100,1.497400,84.446000,3505600000000000.000000,2242300000.000000,0.896300,4.674900,17713.000000,9003.100000,-4.354600,0.254100,6.919100,183240000000.000000,9.651000,32800.000000,1480.600000,23006000000.000000,44.051000,205.690000,4295.300000,13.388000,0.468430,754.610000,83.233000,1.189000,29.550000,7343.700000,0.998150,48777000000000.000000,1.270800,-0.000969,5.295200,6779.000000,227.720000,34.342000,0.340300,0.143370,0.049276,1903200000.000000,0.976730,-56.758000,4.168400,0.348080,4.142000,913.230000,1.246400,7575100000000000.000000,1861.000000,0.283590,0
2,0.178030,-0.006980,907.270000,0.272140,0.459480,0.173270,2298.000000,360650.000000,12245000000000.000000,15827.000000,0.381640,1230300000.000000,0.258070,2.455600,nan,26.873000,7.546300,1.996700,1.952600,817.760000,-2948.700000,2.005400,1.682600,1.196800,74.624000,-32739000000.000000,57189000000.000000,11058.000000,-0.003097,8.024100,1.131800,27940.000000,862460.000000,-0.002207,58491000000000.000000,-897.840000,nan,1.356100,3063.400000,0.086232,16.106000,0.001481,11.476000,5.343000,0.012162,4.101800,-0.882700,8.122800,-0.676690,0.337700,-1.073200,4097.000000,13.458000,159.240000,0.322300,0.560090,0.000455,-0.160830,3.575300,0.609700,0.028301,527130000.000000,14.454000,0.115490,14605.000000,36.992000,-9.639100,64.267000,nan,0.992780,2.589100,430.400000,-44535000000000.000000,5144900000000.000000,0.099591,6.551600,1887.500000,43319.000000,4.

In [4]:
test_data.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
0,0.165850,0.487050,1295.000000,0.023100,0.319000,0.901880,573.290000,3743.700000,2705700000000.000000,6221.000000,1.172000,5175000000.000000,0.258310,3.420500,0.044983,1001.600000,10.104000,10.536000,19.517000,-1.120900,1577.600000,2.184700,96.109000,1.153500,95.865000,2847100000000.000000,96421000000.000000,6406000.000000,0.002990,6.635300,-0.015913,21723.000000,67604.000000,-0.000227,60462000000000000.000000,203.580000,301.790000,1.346100,5524.600000,-0.003163,21.883000,0.027383,7.211300,24.825000,-0.001420,4.895000,-1.087100,7.056100,-1.061500,0.000100,1.856000,449.990000,-0.362860,160.630000,0.248010,0.623000,0.000116,-0.393840,1.903800,0.068588,0.020439,2506500000.000000,48.164000,-0.002272,78643.000000,112.360000,1021.900000,11.481000,1.157300,0.002995,1.819200,1096.800000,300410000000000.000000,2042300000.000000,0.085500,4.631200,1683.100000,9833.600000,nan,0.510410,4.032200,247790000000.000000,69.264000,5494800.000000,1239.000000,534930000.000000,5.308600,182.590000,829.170000,5.231900,0.010301,3640.800000,112.120000,-0.434420,11.161000,3755.700000,0.999260,53382000000.000000,1.386700,-0.001878,58.794000,75657.000000,187.240000,1258.900000,0.163340,0.055398,0.020002,795640000.000000,0.162530,-22.189000,2.065500,0.430880,-10.741000,81606.000000,1.194000,198040000000000.000000,2017.100000,0.463570
1,0.129650,0.373480,1763.000000,0.728840,0.332470,-1.263100,875.550000,554370.000000,595570000000000.000000,934.430000,1.897900,4905100000.000000,0.284710,2.138700,-0.034328,35.223000,6.587300,12.734000,29.590000,4.163900,31915.000000,3.115800,109.430000,-0.852200,991.370000,4700100000.000000,1618200000000.000000,967810.000000,1.021500,2.961700,0.810310,415570.000000,384720.000000,0.002243,6712900000000000.000000,228.590000,33.952000,1.493800,2067.500000,-0.002343,32.772000,0.092283,6.536300,2.077300,0.046237,4.459000,-0.840920,4.447300,-1.001200,0.003750,99.647000,8146.800000,22.504000,153.400000,0.430760,0.743010,-0.000047,-0.116920,1.654400,0.467440,0.129650,444280000.000000,38.506000,0.396220,74296.000000,93.629000,-15.691000,30.655000,1.231100,1.004400,2.248400,780.870000,121360000000000.000000,348820000000.000000,-0.000792,6.999300,2451.600000,10483.000000,-5.354400,0.067286,5.404400,246260000000.000000,810.300000,860670.000000,2165.200000,271150000.000000,10.232000,515.410000,262.710000,5.635700,0.374220,420.160000,116.230000,1.269800,0.786190,1207.000000,1.000700,20650000000000.000000,1.288800,0.248850,9.985700,16323.000000,244.460000,647.580000,0.253820,0.008800,0.018479,112060000.000000,0.815280,-1.634200,1.573600,-1.071200,11.832000,90114.000000,1.150700,43880000000000000.000000,6638.900000,0.281250
2,0.120190,0.445210,736.260000,0.046150,0.296050,0.316650,2659.500000,317140.000000,397780000000000.000000,131.810000,-0.484040,3800000.000000,0.371730,0.402450,0.015119,1991.600000,6.005000,0.991900,11.949000,888.070000,887.700000,3.523700,108.140000,-1.654200,528.140000,2463600000000.000000,9005900000.000000,7259.300000,1.007400,2.612300,1.062500,-208010.000000,65708.000000,0.001391,81298000000000.000000,-84.032000,nan,1.755600,4729.200000,0.003527,861.660000,0.902670,9.393400,58.785000,0.007244,5.664500,-0.627470,6.580600,-1.128200,0.386500,56.547000,125.360000,12.703000,147.380000,0.414370,0.519980,0.000857,-0.120650,5.411500,0.772530,0.128850,5664900000.000000,50.894000,0.378710,80045.000000,65.224000,780.210000,14.125000,1.165700,-0.001285,1.066300,432.010000,-595310000000.000000,1383400000000.000000,-0.006236,5.605200,866.390000,1065.200000

In [5]:
features = [col for col in train_data.columns if col != target]

len(features)

118

# Missing Value

In [6]:
train_data['n_missing'] = train_data[features].isna().sum(axis=1)
test_data['n_missing'] = test_data[features].isna().sum(axis=1)

features.append('n_missing')

In [7]:
modes = train_data[features].mode().iloc[0]

train_data[features] = train_data[features].fillna(modes)
test_data[features] = test_data[features].fillna(modes)

# Scaler 
## Standardize features by removing the mean and scaling to unit variance

In [8]:
train_data.describe()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim,n_missing
count,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000
mean,0.090361,0.348394,4024.055132,0.200181,0.305243,-0.088305,1629.175187,372787.236353,1808771838939572.000000,5254.808445,0.736882,1834241701.218232,0.248605,7.049095,0.020712,468.367326,6.893046,4.436377,22.290407,200.682202,62023.044055,2.269389,87.410766,0.355247,424.728923,3603114521180.718750,1267294935213.164795,2244633.225811,0.339991,7.925109,0.404137,134055.630828,368534.059121,-0.000021,28979198044250788.000000,180.320106,415.657405,1.763374,1966.065066,0.369676,468.807788,0.369887,6.941763,29.499724,0.013411,4.277867,0.045345,6.437406,-0.434872,0.300471,55.947877,2669.793419,11.987613,137.231640,0.251903,0.420606,-0.000007,-0.340153,3.040365,0.552924,0.271104,2446945627.809109,37.854061,0.211090,48344.311096,84.365153,618.529091,28.707851,1.211459,0.427657,1.536854,492.588334,800892494176804.625000,1139692875580.187988,0.386662,6.878436,10734.007629,10532.830538,1.694608,0.195104,3.204542,105531433077.353088,150.524687,6285035.241819,645.226680,33631074571.287983,26.370013,206.524901,3907.128556,6.789791,0.368930,4806.106043,131.772851,0.828505,13.071178,3964.292198,0.999974,14102679069790.107422,1.677109,0.435189,20.051832,332492.515320,582.321467,3810.514783,0.176790,0.160179,0.014146,1670617523.666425,0.396502,-19.848534,2.066367,25.163430,1.881166,63877.509304,1.208017,42310048978034656.000000,4056.395922,0.566662,0.498492,1.900768
std,0.043234,0.146345,6373.280131,0.210931,0.144199,2.110587,1267.576277,344359.156116,2316641258090114.500000,10002.649732,1.484562,2109596231.937814,0.100553,6.580398,0.101573,652.709565,1.701868,3.892431,14.544017,279.788018,89441.646220,0.888829,37.098915,1.633755,321.372738,5876021052637.762695,1632665256904.760254,2287506.502147,0.438771,5.901396,0.779315,202028.777759,466337.782900,0.001527,35428569976242160.000000,697.293529,321.11

In [9]:
train_data['min_row'] = train_data[features].min(axis=1)
train_data['max_row'] = train_data[features].max(axis=1)
train_data['mean_row'] = train_data[features].mean(axis=1)
train_data['std_row'] = train_data[features].std(axis=1)

test_data['min_row'] = test_data[features].min(axis=1)
test_data['max_row'] = test_data[features].max(axis=1)
test_data['mean_row'] = test_data[features].mean(axis=1)
test_data['std_row'] = test_data[features].std(axis=1)

features += ['min_row', 'max_row', 'mean_row', 'std_row']

In [10]:
scaler = preprocessing.StandardScaler()

train_data[features] = scaler.fit_transform(train_data[features])
test_data[features] = scaler.transform(test_data[features])

In [11]:
train_data.describe()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim,n_missing,min_row,max_row,mean_row,std_row
count,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000
mean,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.498492,-0.000000,0.000000,0.000000,0.000000,-0.000000
std,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.0000

## Setting Parameters

In [12]:
xgb_params = {
    'n_estimators' : 1000,
    'reg_lambda' : 3,
    'reg_alpha' : 26,
    'subsample' : 0.6000000000000001,
    'colsample_bytree' : 0.6000000000000001,
    'max_depth' : 9,
    'min_child_weight' : 5,
    'gamma' : 13.054739572819486,
    'learning_rate': 0.02,
    'tree_method': 'gpu_hist',
    'booster': 'gbtree'
}

lgbm_params = {
    "objective": "binary",
    "learning_rate": 0.01,
    'n_estimators': 1000,
    'num_leaves': 184,
    'min_child_samples': 63,
    'feature_fraction': 0.6864594334728974,
    'bagging_fraction': 0.9497327922401265,
    'bagging_freq': 1,
    'reg_alpha': 19,
    'reg_lambda': 19,
}

catb_params = {
    'iterations': 1000, 
    'objective': 'CrossEntropy', 
    'bootstrap_type': 'Bernoulli', 
    'od_wait': 1144, 
    'learning_rate': 0.023575206684596582, 
    'reg_lambda': 36.30433203563295, 
    'random_strength': 43.75597655616195, 
    'depth': 7, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 1, 
    'subsample': 0.8227911142845009,
    'devices' : '0',
    'verbose' : 0
}

# Setting the Cross Validate and Train Model

### Model XGBC

In [13]:
model = XGBClassifier(**xgb_params)
model

XGBClassifier(base_score=None, booster='gbtree', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.6000000000000001,
              gamma=13.054739572819486, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.02,
              max_delta_step=None, max_depth=9, min_child_weight=5, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, reg_alpha=26,
              reg_lambda=3, scale_pos_weight=None, subsample=0.6000000000000001,
              tree_method='gpu_hist', validate_parameters=None, verbosity=None)

In [14]:
X = train_data[features].copy()
y = TARGET
X_test = test_data.copy()

In [15]:
y = TARGET

X.describe()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,n_missing,min_row,max_row,mean_row,std_row
count,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000,957919.000000
mean,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000
std,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001

In [17]:
splits = 5
skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)

oof_preds = np.zeros((X.shape[0],))
preds = 0
model_fi = 0
total_mean_auc = 0

for num, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
    
    imputer = SimpleImputer(strategy='median')
    X_train = imputer.fit_transform(X_train)
    X_valid = imputer.transform(X_valid)
    
    model.fit(X_train, y_train,
              verbose=False,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="auc",
              early_stopping_rounds=300,
              )
    
    X_test = imputer.transform(X_test)
    
    preds += model.predict_proba(X_test)[:, 1] / splits
   
   
    
    model_fi += model.feature_importances_ / splits
    
    oof_preds[valid_idx] = model.predict_proba(X_valid)[:, 1]
    
    fold_auc = roc_auc_score(y_valid, oof_preds[valid_idx])
    print(f"Fold {num} ROC AUC: {fold_auc}")

    total_mean_auc += fold_auc / splits
    
print(f"\nOverall ROC AUC: {total_mean_auc}")


Fold 0 ROC AUC: 0.8121415899435734
Fold 1 ROC AUC: 0.8140037311039028
Fold 2 ROC AUC: 0.8127198823661544
Fold 3 ROC AUC: 0.8125704520095638
Fold 4 ROC AUC: 0.8121161474994273

Overall ROC AUC: 0.8127103605845243


In [18]:
XGBC_score = total_mean_auc
XGBC_score

0.8127103605845243

### Submission

In [19]:
submission.claim = preds
submission.head()


,id,claim
0,957919,0.616455
1,957920,0.117593
2,957921,0.615548
3,957922,0.139750
4,957923,0.142130


In [20]:
submission.to_csv('submissionXGBC.csv', index=False)

### LGBMC Model

In [21]:
model = LGBMClassifier(**lgbm_params)
model

LGBMClassifier(bagging_fraction=0.9497327922401265, bagging_freq=1,
               feature_fraction=0.6864594334728974, learning_rate=0.01,
               min_child_samples=63, n_estimators=1000, num_leaves=184,
               objective='binary', reg_alpha=19, reg_lambda=19)

In [22]:
splits = 5
skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)

oof_preds = np.zeros((X.shape[0],))
preds = 0
model_fi = 0
total_mean_auc = 0

for num, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
    
    imputer = SimpleImputer(strategy='median')
    X_train = imputer.fit_transform(X_train)
    X_valid = imputer.transform(X_valid)
    
    model.fit(X_train, y_train,
              verbose=False,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="auc",
              early_stopping_rounds=300,
              )
    
    X_test = imputer.transform(X_test)
    
    preds += model.predict_proba(X_test)[:, 1] / splits
   
   
    
    model_fi += model.feature_importances_ / splits
    
    oof_preds[valid_idx] = model.predict_proba(X_valid)[:, 1]
    
    fold_auc = roc_auc_score(y_valid, oof_preds[valid_idx])
    print(f"Fold {num} ROC AUC: {fold_auc}")

    total_mean_auc += fold_auc / splits
    
print(f"\nOverall ROC AUC: {total_mean_auc}")

Fold 0 ROC AUC: 0.8133090682871394
Fold 1 ROC AUC: 0.8145690700399486
Fold 2 ROC AUC: 0.8134982575316991
Fold 3 ROC AUC: 0.8136227397728083
Fold 4 ROC AUC: 0.8127641606316659

Overall ROC AUC: 0.8135526592526523


In [23]:
LGB_Score = total_mean_auc

#### Submission

In [24]:
submission = pd.read_csv("sample_solution.csv")

In [25]:
submission.claim = preds
submission.head()

,id,claim
0,957919,0.629321
1,957920,0.116280
2,957921,0.624240
3,957922,0.137937
4,957923,0.144077


In [26]:
submission.to_csv('submissionLGBMC.csv', index=False)



### CatBoostClassifier Model

In [27]:
model = CatBoostClassifier(**catb_params)
model

In [28]:

skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)

oof_preds = np.zeros((X.shape[0],))
preds = 0
model_fi = 0
total_mean_auc = 0

for num, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
    
    imputer = SimpleImputer(strategy='median')
    X_train = imputer.fit_transform(X_train)
    X_valid = imputer.transform(X_valid)
    
    model.fit(X_train, y_train,
              verbose=False,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              early_stopping_rounds=300,
              )
    
    X_test = imputer.transform(X_test)
    
    preds += model.predict_proba(X_test)[:, 1] / splits
   
   
    
    model_fi += model.feature_importances_ / splits
    
    oof_preds[valid_idx] = model.predict_proba(X_valid)[:, 1]
    
    fold_auc = roc_auc_score(y_valid, oof_preds[valid_idx])
    print(f"Fold {num} ROC AUC: {fold_auc}")

    total_mean_auc += fold_auc / splits
    
print(f"\nOverall ROC AUC: {total_mean_auc}")

Fold 0 ROC AUC: 0.8122416733812063
Fold 1 ROC AUC: 0.8138562050579907
Fold 2 ROC AUC: 0.8125555969368421
Fold 3 ROC AUC: 0.8127739362803434
Fold 4 ROC AUC: 0.8116527092631229

Overall ROC AUC: 0.8126160241839011


In [29]:
CAT_Score = total_mean_auc

In [30]:
#### Submission

In [31]:
submission = pd.read_csv("sample_solution.csv")

In [32]:
submission.claim = preds
submission.head()

,id,claim
0,957919,0.597370
1,957920,0.122496
2,957921,0.594329
3,957922,0.135650
4,957923,0.143358


In [33]:
submission.to_csv('submissionCAT.csv', index=False)